In [ ]:
import mne
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse

# Load the sample data
data_path = sample.data_path()
raw = mne.io.read_raw_fif(data_path + '/MEG/sample/sample_audvis_raw.fif', preload=True)

# Find events
events = mne.find_events(raw)
event_id = {'Auditory/Left': 1}
epochs = mne.Epochs(raw, events, event_id, tmin=-0.1, tmax=0.4, baseline=(None, 0), preload=True)

# Compute the evoked response (averaging the epochs)
evoked = epochs.average()

# Load the forward solution
fwd = mne.read_forward_solution(data_path + '/MEG/sample/sample_audvis-meg-eeg-oct-6-fwd.fif')

# Load the noise covariance matrix
noise_cov = mne.read_cov(data_path + '/MEG/sample/sample_audvis-cov.fif')

# Compute the inverse operator
inverse_operator = make_inverse_operator(evoked.info, fwd, noise_cov, loose=0.2, depth=0.8)

# Apply the inverse operator to get the source estimate
method = "MNE"  # You can also try sLORETA, dSPM, eLORETA
snr = 3.0
lambda2 = 1.0 / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None)

# Visualize the source estimate on the 3D brain model
brain = stc.plot(hemi='both', subjects_dir=data_path + '/subjects', subject='sample', time_viewer=True, transparent=True)
